In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
! pip install imutils 
from imutils import paths
import os
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import random
import warnings
warnings.filterwarnings('ignore')
from tensorflow import keras

  Preparing metadata (setup.py) ... - done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25858 sha256=3deb89aae190d3ed9d45cb7d7455b5cb9f2e2aeac5154c096bb8203923907563
  Stored in directory: /root/.cache/pip/wheels/86/d7/0a/4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils


In [2]:
COVID_PATH_1 = "../input/chest-xray-for-covid19-detection/Dataset/Train/Covid"
NORMAL_PATH_1 = "../input/chest-xray-for-covid19-detection/Dataset/Train/Normal"

In [3]:
COVID_PATH_2 = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID"
NORMAL_PATH_2 = "../input/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal"

In [4]:
def sp_noise(image,prob):
    '''
    As the name suggests salt (white) in pepper (black)–white spots 
    in the dark regions or pepper (black) in salt (white)–black spots
    in the white regions. In other words, an image having salt-and-pepper 
    noise will have a few dark pixels in bright regions and a few bright pixels
    in dark regions.
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

In [5]:
def add1(path,array):
    for item in os.listdir(path):
        img = cv2.imread(os.path.join(path, item))
        
        img = cv2.resize(img, (224, 224))
        img=sp_noise(img,0.7)
        array.append(img)
        
def add2(path,array):
    for item in os.listdir(path):
        img = cv2.imread(os.path.join(path, item))
        
        img = cv2.resize(img, (224, 224))
        array.append(img)

In [6]:
dataset_1=[]

add1(COVID_PATH_1,dataset_1)
add1(NORMAL_PATH_1,dataset_1)


In [7]:
dataset_2=[]
add2(COVID_PATH_2,dataset_2)
add2(NORMAL_PATH_2,dataset_2)


In [8]:
type(dataset_2[0])

numpy.ndarray

In [9]:
print(len(dataset_1))
print(len(dataset_2))

val_1 = random.sample(dataset_1, 25)
val_2 = random.sample(dataset_2, 25)

for i in range(25):
    val_1[i]=[val_1[i],1]
    
for i in range(25):
    val_2[i]=[val_2[i],2]
    
validation=val_1+val_2

np.random.shuffle(validation)


288
13808


In [10]:
label=[]
for ele in validation:
    label.append(ele[1])

In [11]:

def mean_dataset(dataset):
    count = 0
    mean = 0

    for image in dataset:
        val = np.reshape(image[:,:,:], -1)
        img_mean = np.mean(val)
        mean=(mean * count + img_mean) /(count + 1)
        count+=1
    return mean

def mean_image(image):
    val = np.reshape(image[:,:,0], -1)
    return np.mean(val)

In [12]:
mean_dataset_1=mean_dataset(dataset_1)
mean_dataset_2=mean_dataset(dataset_2)

print(mean_dataset_1)
print(mean_dataset_2)

76.53685589225923
131.92409118945366


In [13]:
def choose_model(test,mean_dataset_1,mean_dataset_2):
    mean_current_image=mean_image(test)
    if abs(mean_current_image-mean_dataset_1)<abs(mean_current_image-mean_dataset_2):
        return 1
    else:
        return 2

In [14]:
model_1 = keras.models.load_model('../input/dataset-models/models/model_I.h5')
model_2 = keras.models.load_model('../input/dataset-models/models/model_II.h5')

2022-04-20 19:53:29.045253: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 19:53:29.145318: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 19:53:29.146209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-20 19:53:29.147857: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [15]:
def run_model(model_1,model_2,test,pred):
    
    test=test.reshape((-1,224,224,3))
    if choose_model(test,mean_dataset_1,mean_dataset_2)==1:
        pred.append(1)
        print('Using Model_1')
        res=(int(model_1.predict(test)))
        if res==0:
            print('Covid')
        else:
            print('Normal')
    else:
        pred.append(2)
        print('Using Model_2')
        res=(np.argmax(model_2.predict(test)))
        if res==0:
            print('Covid')
        else:
            print('Normal')

In [16]:
type(validation[1][0])

numpy.ndarray

In [17]:
pred=[]
for ele in validation:
    run_model(model_1,model_2,ele[0],pred)
    
    
from sklearn.metrics import accuracy_score
accuracy_score(label, pred)

Using Model_1


2022-04-20 19:53:42.790576: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-04-20 19:53:43.643697: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_2
Normal
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_2
Normal
Using Model_1
Covid
Using Model_1
Normal
Using Model_1
Normal
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_2
Normal
Using Model_1
Covid
Using Model_1
Normal
Using Model_1
Covid
Using Model_2
Normal
Using Model_1
Covid
Using Model_1
Normal
Using Model_1
Normal
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Normal
Using Model_1
Covid
Using Model_1
Covid
Using Model_2
Normal
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_1
Covid
Using Model_2
Normal
Using Model_1
Covid
Using Model_1
Covid
Using Model_2
Normal


0.64